(always be aware of your imports and <b><u><i>preserve namespaces</i></u></b>!!!)

In [1]:
import os
import sys
import time
import numpy as np
import scipy.ndimage as nd
import matplotlib.pyplot as plt

%matplotlib tk
plt.ion()
plt.rcParams["image.cmap"] = "gist_gray"

---

## Regression and the normal equation

### Linear fit

Here are two data sets:

In [2]:
A = 1.2
B = 0.34

x = np.linspace(-5.0 * np.pi, 5.0 * np.pi, 100, False)
y0 = np.cos(x)
y1 = A * y0 + B

In [3]:
fig0, ax0 = plt.subplots(num=0)
lin0a, = ax0.plot(x, y0)
lin0b, = ax0.plot(x, y1)
fig0.canvas.draw()

Our task is to figure out the amplitude ($A$) and offset ($B$) parameters from the data alone.  We are trying to solve the equation,

$y_1 = A \times y_0 + B$

But even though it looks like this is one equation, it is actually 100,

$y_1[0] = A \times y_0[0] + B$<br>
$y_1[1] = A \times y_0[1] + B$<br>
$y_1[2] = A \times y_0[2] + B$<br>
$.$<br>
$.$<br>
$.$<br>
$y_1[99] = A \times y_0[99] + B$<br>

which can be written as

$\left(\begin{array}{c}
y_1[0] \\
y_1[1] \\
y_1[2] \\
. \\
. \\
. \\
y_1[99]
\end{array}\right)
=
\left(\begin{array}{cc}
y_0[0] & 1 \\
y_0[1] & 1 \\
y_0[2] & 1 \\
. & . \\
. & . \\
. & . \\
y_0[99] & 1
\end{array}\right)
\times
\left(\begin{array}{c}
A \\
B
\end{array}\right)
$

or

$\vec{y}_1 = \mathbf{P}\vec{a}$

where $\vec{y}_1$ is still an array (but let's call it a vector now), $\mathbf{P}$ is a "matrix", and $\vec{a}$ is also a vector of coefficients (in our case amplitude and offset).  Equations that look like the one above are called the <b>Normal Equation</b>.

You might think that you can just solve for $\vec{a}$ by doing

$\vec{a} = \frac{\vec{y}_1}{\mathbf{P}}$

but $1/\mathbf{P}$ has no real meaning since $\mathbf{P}$ is a matrix.  To find the actual solution, we need to first define the transpose,

$\mathbf{P}^T = 
\left(\begin{array}{ccccccc}
y_0[0] & y_0[1] & y_0[2] & . & . & . & y_0[99] \\
1 & 1 & 1 & . & . & . & 1 \\
\end{array}\right)$

we solve the normal equation by first multiplying both sides by the transpose of $\mathbf{P}$

$(\mathbf{P}^T\vec{y}_1) = (\mathbf{P}^T\mathbf{P})\vec{a}$

and then multiplying by inverse of that matrix product:

$(\mathbf{P}^T\mathbf{P})^{-1}(\mathbf{P}^T\vec{y}_1) = \vec{a}$

Note that the matrix inverse is not just replacing all elements of a matrix by 1 over its value, nor is matrix multiplication just doing elementwise multiplication.  It is a bit more complicated and beyond the present scope, but you can read about them <a href="http://en.wikipedia.org/wiki/Matrix_multiplication#Matrix_product_.28two_matrices.29">here</a> and <a href="http://en.wikipedia.org/wiki/Invertible_matrix#Inversion_of_2.C3.972_matrices">here</a> (note this last link is for $2 \times 2$ matrices, which is our case in this example).

Fortunately, python has a linear algebra model that will do these calculations for you.

In [4]:
PT = np.vstack([y0, np.ones(len(y0))])
P  = PT.transpose()
print("P = \n{0}".format(P))

P = 
[[ -1.00000000e+00   1.00000000e+00]
 [ -9.51056516e-01   1.00000000e+00]
 [ -8.09016994e-01   1.00000000e+00]
 [ -5.87785252e-01   1.00000000e+00]
 [ -3.09016994e-01   1.00000000e+00]
 [  5.51091060e-16   1.00000000e+00]
 [  3.09016994e-01   1.00000000e+00]
 [  5.87785252e-01   1.00000000e+00]
 [  8.09016994e-01   1.00000000e+00]
 [  9.51056516e-01   1.00000000e+00]
 [  1.00000000e+00   1.00000000e+00]
 [  9.51056516e-01   1.00000000e+00]
 [  8.09016994e-01   1.00000000e+00]
 [  5.87785252e-01   1.00000000e+00]
 [  3.09016994e-01   1.00000000e+00]
 [ -4.28626380e-16   1.00000000e+00]
 [ -3.09016994e-01   1.00000000e+00]
 [ -5.87785252e-01   1.00000000e+00]
 [ -8.09016994e-01   1.00000000e+00]
 [ -9.51056516e-01   1.00000000e+00]
 [ -1.00000000e+00   1.00000000e+00]
 [ -9.51056516e-01   1.00000000e+00]
 [ -8.09016994e-01   1.00000000e+00]
 [ -5.87785252e-01   1.00000000e+00]
 [ -3.09016994e-01   1.00000000e+00]
 [  3.06161700e-16   1.00000000e+00]
 [  3.09016994e-01   1.00000000e+

In [5]:
PTPinv = np.linalg.inv(np.dot(PT, P))
PTy1 = np.dot(PT, y1)

print("PTPinv = \n{0}\nPTy1 = {1}".format(PTPinv, PTy1))

PTPinv = 
[[  2.00000000e-02  -7.77156117e-19]
 [ -7.77156117e-19   1.00000000e-02]]
PTy1 = [ 60.  34.]


And so finally:

In [6]:
avec = np.dot(PTPinv, PTy1)
print("avec = {0}\ninput:\n  A={1}, B={2}".format(avec, A, B))

avec = [ 1.2   0.34]
input:
  A=1.2, B=0.34


It turns out that the solution to the normal equation is equivalent to the classic "least squares minimization" solution.

In [6]:
plt.close(0)

### Multivariate regression

Before we start applying this technique to images, let's look at an example that isn't so trivial.  First, let's add some noise and regress several "template" functions:

In [7]:
xx = np.linspace(0, 10, 100, False)

func1 = np.exp(-xx**2 / (2.0 * 4.0**2))
func2 = np.cos(xx)
func3 = np.sqrt(xx / 20.)

np.random.seed(712)
noise = np.random.randn(len(xx))

yy = -5 * func1 + 3 * func2 + 7 * func3 + 2.0 * noise

plt.close(1)
fig1, ax1 = plt.subplots(num=1)
lin1a, = ax1.plot(xx, yy, 'o')
fig1.canvas.draw()

In [8]:
PT     = np.vstack([func1, func2, func3])
P      = PT.transpose()
PTPinv = np.linalg.inv(np.dot(PT, P))
PTyy   = np.dot(PT, yy)
avec   = np.dot(PTPinv, PTyy)

print("avec = {0}".format(avec))

lin1b, = ax1.plot(xx, np.dot(P, avec), color="darkred")
fig1.canvas.draw()

avec = [-5.11200294  2.68070448  7.1369897 ]


But now, what if we try the **completely** wrong model?

In [9]:
PT     = np.vstack([xx**4, xx**3, xx**2, xx, np.ones(len(xx))])
P      = PT.transpose()
PTPinv = np.linalg.inv(np.dot(PT, P))
PTyy   = np.dot(PT, yy)
avec   = np.dot(PTPinv, PTyy)

print("avec = {0}".format(avec))

lin1c, = ax1.plot(xx,np.dot(P, avec), color="black")
lin1b.set_visible(False)
fig1.canvas.draw()

avec = [ 0.01000133 -0.28422094  2.45504191 -6.13329667  0.54251651]


Pretty good for the wrong model!  So regressions (and models in general) can be powerful, but in order to evaluate a fit, you really have to know something about your errors.

In [10]:
[plt.close(i) for i in range(2)]

[None, None]

---

## Correlation and handwritten digits

*This will be only the **most basic** technique of recognizing handwritten characters.  There are much more effective ways to solve this problem, but it is a good illustration of regressions and correlations...*

First let's load the handwritten digits file:

In [11]:
img = nd.imread("images/digits.png")
nrow, ncol = img.shape[0:2]
xs = 10.
ys = xs * float(nrow) / float(ncol)

plt.close(0)
fig0, ax0 = plt.subplots(num=0, figsize=[xs, ys])
fig0.subplots_adjust(0, 0, 1, 1)
ax0.axis("off")
im0 = ax0.imshow(img)
fig0.canvas.draw()

We'd like to isolate the numbers from this image.  To do that, we use some reshaping and transposing trickery...

In [12]:
nums = img.reshape(50, 20, 100, 20) \
    .transpose(0, 2, 1, 3) \
    .reshape(5000, 20, 20)

and we have each number isolated:

In [13]:
plt.close(1)
fig1, ax1 = plt.subplots(num=1, figsize=[xs/2, xs/2])
fig1.subplots_adjust(0, 0, 1, 1)
ax1.axis("off")
im1 = ax1.imshow(nums[0])
fig1.canvas.draw()

In [14]:
t0 = time.time()
dt = 0.0
np.random.seed(712)
while dt < 20.:
    ii = int(np.floor(len(nums) * np.random.rand()))
    im1.set_data(nums[ii])
    fig1.canvas.draw()
    time.sleep(0.5)
    dt = time.time() - t0

We can look at the average for a given digit:

In [15]:
nums_avg = np.array([nums[i*500:(i + 1)*500].mean(0) for i 
                     in range(10)])

for ii in range(10):
    im1.set_data(nums_avg[ii])
    fig1.canvas.draw()
    time.sleep(2.0)

Let's use these averages as "templates" for a regression against an individual sample:

In [36]:
index =4014
samp  = nums[index]

PT     = nums_avg.reshape(10, 400)
P      = PT.transpose()
PTPinv = np.linalg.inv(np.dot(PT, P))
PTyy   = np.dot(PT, samp.flatten())
avec   = np.dot(PTPinv, PTyy)

print("avec = {0}\n".format(avec.round(2)))
print("Therefore my guess is that this is an " + 
      "image of the number {0}".format(avec.argmax()))

avec = [ 0.04  0.37 -0.04 -0.2  -0.11 -0.27 -0.08  0.05  1.36 -0.24]

Therefore my guess is that this is an image of the number 8


In [37]:
im1.set_data(samp)
try:
    lab
except:
    lab = ax1.text(0, 0, "Guess: ", va="top", fontsize=20, 
                   color="w")
lab.set_text('Guess: {0}'.format(avec.argmax()))
fig1.canvas.draw()

---

In [19]:
plt.close("all")